<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/vector_stores/ElasticsearchIndexDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elasticsearch Vector Store

Elasticsearch is a distributed, RESTful search and analytics engine built on top of Apache Lucene. It offers different retrieval options including dense vector retrieval, sparse vector retrieval, keyword search and hybrid search.

[Sign up](https://cloud.elastic.co/registration?utm_source=llama-index&utm_content=documentation) for a free trial of Elastic Cloud or run a local server like described below.

Requires Elasticsearch 8.9.0 or higher and AIOHTTP.

In [ ]:
%pip install -qU llama-index-vector-stores-elasticsearc llama-index openai

In [ ]:
import getpass
import os

import openai

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

openai.api_key = os.environ["OPENAI_API_KEY"]

## Running and connecting to Elasticsearch
Two ways to setup an Elasticsearch instance for use with:

### Elastic Cloud
Elastic Cloud is a managed Elasticsearch service. [Sign up](https://cloud.elastic.co/registration?utm_source=llama-index&utm_content=documentation) for a free trial.

### Locally
Get started with Elasticsearch by running it locally. The easiest way is to use the official Elasticsearch Docker image. See the Elasticsearch Docker documentation for more information.

```bash
docker run -p 9200:9200 \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "xpack.license.self_generated.type=trial" \
  docker.elastic.co/elasticsearch/elasticsearch:8.13.2
```

## Configuring ElasticsearchStore
The ElasticsearchStore class is used to connect to an Elasticsearch instance. It requires the following parameters:

        - index_name: Name of the Elasticsearch index. Required.
        - es_client: Optional. Pre-existing Elasticsearch client.
        - es_url: Optional. Elasticsearch URL.
        - es_cloud_id: Optional. Elasticsearch cloud ID.
        - es_api_key: Optional. Elasticsearch API key.
        - es_user: Optional. Elasticsearch username.
        - es_password: Optional. Elasticsearch password.
        - text_field: Optional. Name of the Elasticsearch field that stores the text.
        - vector_field: Optional. Name of the Elasticsearch field that stores the
                    embedding.
        - batch_size: Optional. Batch size for bulk indexing. Defaults to 200.
        - distance_strategy: Optional. Distance strategy to use for similarity search.
                        Defaults to "COSINE".

### Example: Connecting locally
```python
from llama_index.vector_stores import ElasticsearchStore

es = ElasticsearchStore(
    index_name="my_index",
    es_url="http://localhost:9200",
)
```

### Example: Connecting to Elastic Cloud with username and password

```python
from llama_index.vector_stores import ElasticsearchStore

es = ElasticsearchStore(
    index_name="my_index",
    es_cloud_id="<cloud-id>", # found within the deployment page
    es_user="elastic"
    es_password="<password>" # provided when creating deployment. Alternatively can reset password.
)
```

### Example: Connecting to Elastic Cloud with API Key

```python
from llama_index.vector_stores import ElasticsearchStore

es = ElasticsearchStore(
    index_name="my_index",
    es_cloud_id="<cloud-id>", # found within the deployment page
    es_api_key="<api-key>" # create an API key within Kibana (Security -> API Keys)
)
```


#### Example data

In [ ]:
from llama_index.core.schema import TextNode

movies = [
    TextNode(
        text="The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.",
        metadata={"title": "Pulp Fiction"},
    ),
    TextNode(
        text="When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.",
        metadata={"title": "The Dark Knight"},
    ),
    TextNode(
        text="An insomniac office worker and a devil-may-care soapmaker form an underground fight club that evolves into something much, much more.",
        metadata={"title": "Fight Club"},
    ),
    TextNode(
        text="A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into thed of a C.E.O.",
        metadata={"title": "Inception"},
    ),
    TextNode(
        text="A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers.",
        metadata={"title": "The Matrix"},
    ),
    TextNode(
        text="Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.",
        metadata={"title": "Se7en"},
    ),
    TextNode(
        text="An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.",
        metadata={"title": "The Godfather", "theme": "Mafia"},
    ),
]

## Retrieval Examples

This section shows the different retrieval options available through the `ElasticsearchStore` and make use of them via a VectorStoreIndex.

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.elasticsearch import ElasticsearchStore

We first define a helper function to retrieve and print results for user query input:

In [ ]:
def print_results(results):
    for rank, result in enumerate(results, 1):
        print(
            f"{rank}. title={result.metadata['title']} score={result.get_score()} text={result.get_text()}"
        )


def search(
    vector_store: ElasticsearchStore, nodes: list[TextNode], query: str
):
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(nodes, storage_context=storage_context)

    print(">>> Documents:")
    retriever = index.as_retriever()
    results = retriever.retrieve(query)
    print_results(results)

    print("\n>>> Answer:")
    query_engine = index.as_query_engine()
    response = query_engine.query(query)
    print(response)

### Dense retrieval

Here we use embeddings from OpenAI to search.

In [ ]:
from llama_index.vector_stores.elasticsearch import AsyncDenseVectorStrategy

dense_vector_store = ElasticsearchStore(
    es_url="http://localhost:9200",  # for Elastic Cloud authentication see above
    index_name="movies_dense",
    retrieval_strategy=AsyncDenseVectorStrategy(),
)

search(dense_vector_store, movies, "which movie involves dreaming?")

>>> Documents:
1. title=Inception score=1.0 text=A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into thed of a C.E.O.

>>> Answer:
Inception


This is also the default retrieval strategy:

In [ ]:
default_store = ElasticsearchStore(
    es_url="http://localhost:9200",  # for Elastic Cloud authentication see above
    index_name="movies_default",
)

search(default_store, movies, "which movie involves dreaming?")

>>> Documents:
1. title=Inception score=1.0 text=A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into thed of a C.E.O.

>>> Answer:
Inception


### Sparse retrieval

For this example you first need to [deploy the ELSER model](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) version two in your Elasticsearch deployment.

In [ ]:
from llama_index.vector_stores.elasticsearch import AsyncSparseVectorStrategy

sparse_vector_store = ElasticsearchStore(
    es_url="http://localhost:9200",  # for Elastic Cloud authentication see above
    index_name="movies_sparse",
    retrieval_strategy=AsyncSparseVectorStrategy(model_id=".elser_model_2"),
)

search(sparse_vector_store, movies, "which movie involves dreaming?")

>>> Documents:
1. title=Inception score=1.0 text=A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into thed of a C.E.O.

>>> Answer:
Inception


### Keyword retrieval

To use classic full-text search, you can use the BM25 strategy.

In [ ]:
from llama_index.vector_stores.elasticsearch import AsyncBM25Strategy

bm25_store = ElasticsearchStore(
    es_url="http://localhost:9200",  # for Elastic Cloud authentication see above
    index_name="movies_bm25",
    retrieval_strategy=AsyncBM25Strategy(),
)

search(bm25_store, movies, "joker")

>>> Documents:
1. title=The Dark Knight score=1.0 text=When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.

>>> Answer:
The Joker is a menacing character who wreaks havoc and chaos on the people of Gotham, posing a significant challenge for Batman to combat injustice.


### Hybrid retrieval

Combining dense retrieval and keyword search for hybrid retrieval can be enabled by setting a flag.

In [ ]:
from llama_index.vector_stores.elasticsearch import AsyncDenseVectorStrategy

hybrid_store = ElasticsearchStore(
    es_url="http://localhost:9200",  # for Elastic Cloud authentication see above
    index_name="movies_hybrid",
    retrieval_strategy=AsyncDenseVectorStrategy(hybrid=True),
)

search(hybrid_store, movies, "which movie involves dreaming?")

>>> Documents:
1. title=Inception score=0.36787944117144233 text=A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into thed of a C.E.O.

>>> Answer:
"Inception" is the movie that involves dreaming.


### Metadata Filters

We can also apply filters to the query engine based on the metadata of our documents.

In [ ]:
from llama_index.core.vector_stores import ExactMatchFilter, MetadataFilters

metadata_store = ElasticsearchStore(
    es_url="http://localhost:9200",  # for Elastic Cloud authentication see above
    index_name="movies_metadata",
)
storage_context = StorageContext.from_defaults(vector_store=metadata_store)
index = VectorStoreIndex(movies, storage_context=storage_context)

# Metadata filter
filters = MetadataFilters(
    filters=[ExactMatchFilter(key="theme", value="Mafia")]
)
retriever = index.as_retriever(filters=filters)

results = retriever.retrieve("What is inception about?")
print_results(results)

1. title=The Godfather score=1.0 text=An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.


## Custom Filters and overriding Query 
The elastic search implementation only supports ExactMatchFilters provided from LlamaIndex at the moment. Elasticsearch itself supports a wide range of filters, including range filters, geo filters, and more. To use these filters, you can pass them in as a list of dictionaries to the `es_filter` parameter.

In [ ]:
def custom_query(query, query_str):
    print("custom query", query)
    return query


query_engine = index.as_query_engine(
    vector_store_kwargs={
        "es_filter": [{"match": {"title": "matrix"}}],
        "custom_query": custom_query,
    }
)
query_engine.query("what is this movie about?")

custom query {'knn': {'filter': [{'match': {'title': 'matrix'}}], 'field': 'embedding', 'k': 2, 'num_candidates': 20, 'query_vector': [0.00446691969409585, -0.038953110575675964, -0.023963095620274544, -0.024891795590519905, -0.016729693859815598, 0.017200583592057228, -0.002360992832109332, -0.012622482143342495, -0.009980263188481331, -0.026108263060450554, 0.02950914017856121, 0.018626336008310318, -0.016154160723090172, -0.012099270708858967, 0.03777588531374931, 0.006209868937730789, 0.03539527207612991, -0.011746102944016457, 0.0029888467397540808, -0.022066453471779823, -0.02290359139442444, -0.011752642691135406, -0.018744058907032013, -0.015251620672643185, 0.0034074161667376757, 0.00014756205200683326, 0.022955913096666336, -0.02264198660850525, 0.002032350515946746, -0.021778685972094536, 0.012164671905338764, -0.015055416151881218, 0.006543416064232588, -0.009509372524917126, -0.008632993325591087, -0.006814832333475351, 0.011765723116695881, -0.01788076013326645, 0.0016669